In [ ]:
#!/usr/bin/env python3

import os

import dask.distributed

import numcodecs

import xarray as xr

 

client = dask.distributed.Client(

    scheduler_file="/home/ad/briolf/scheduler.json")

print(client)

 

root = "/home/ad/briolf/odatis/eNATL60/BLBT02/gridT-2D/"

files = [

    os.path.join(root, item) for item in os.listdir(root)

    if item.endswith(".nc")

]

 

# these are variables I just want to drop forever

drop_vars = ['nav_lat', 'nav_lon', 'somxl010', 'sosaline', 'sosstsst']

 

# these are variables I want to drop while running `open_mfdataset` but then add back later

extra_coord_vars = ['time_counter', 'y', 'x']

extra_coord_vars = []

 

chunks = dict(time_counter=1)

 

open_kwargs = dict(drop_variables=(drop_vars + extra_coord_vars),

                   chunks=chunks,

                   decode_cf=True,

                   decode_times=False,

                   concat_dim="time_counter")  #, combine='nested')

ds = xr.open_mfdataset(files, parallel=True, **open_kwargs)

template = xr.open_dataset(files[0], decode_cf=False, decode_times=False)

ds["nav_lat"] = template["nav_lat"]

ds["nav_lon"] = template["nav_lon"]

ds

del template

 

compressor = numcodecs.Blosc(cname='snappy', clevel=6, shuffle=-1)

encoding = {vname: {'compressor': compressor} for vname in ds.variables}

outdir = '/home/ad/briolf/odatis/briolf/NATL60'

 

ds = ds.chunk(chunks=dict(time_counter=240, y=240, x=480))

 

ds.to_zarr(outdir, encoding=encoding, mode="w")

 

client.close()

cluster.close()
